In [1]:
import sys
sys.path.append('../scripts/')
from kf import *

In [2]:
def make_ax():
    fig = plt.figure(figsize=(4, 4))
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_xlabel("X", fontsize=10)
    ax.set_ylabel("Y", fontsize=10)
    return ax

def draw_trajectory(xs, ax):
    poses = [xs[s] for s in range(len(xs))]
    ax.scatter([e[0] for e in poses], [e[1] for e in poses], s=5, marker=".", color="black")
    ax.plot([e[0] for e in poses], [e[1] for e in poses], linewidth=0.5, color="black")
    
def draw_observation(xs, zlist, ax):
    for s in range(len(xs)):
        if s not in zlist:
            continue
            
        for obs in zlist[s]:
            x, y, theta = xs[s]
            ell, phi = obs[1][0], obs[1][1]
            mx = x + ell*math.cos(theta + phi)
            my = y + ell*math.sin(theta + phi)
            ax.plot([x, mx], [y, my], color="pink", alpha=0.5)

def draw_edges(edges, ax):
    for e in edges:
        ax.plot([e.x1[0], e.x2[0]], [e.x1[1], e.x2[1]], color="red", alpha=0.5)
            
def draw(xs, zlist, edges):
    ax = make_ax()
    draw_observation(xs, zlist, ax)
    draw_edges(edges, ax)
    draw_trajectory(xs, ax)
    plt.show()

In [3]:
def read_data():
    hat_xs = {}
    zlist = {}
    
    with open("log.txt") as f:
        for line in f.readlines():
            tmp = line.rstrip().split()
            
            step = int(tmp[1])
            if tmp[0] == "x":
                hat_xs[step] = np.array([float(tmp[2]), float(tmp[3]), float(tmp[4])]).T
            elif tmp[0] == "z":
                if step not in zlist:
                    zlist[step] = []
                zlist[step].append((int(tmp[2]), np.array([float(tmp[3]), float(tmp[4]), float(tmp[5])]).T))
                
        return hat_xs, zlist

In [4]:
class ObsEdge:
    def __init__(self, t1, t2, z1, z2, xs, sensor_noise_rate=[0.14, 0.05, 0.05]):
        assert z1[0] == z2[0]
        
        self.t1, self.t2 = t1, t2
        self.x1, self.x2 = xs[t1], xs[t2]
        self.z1, self.z2 = z1[1], z2[1]
        
        s1 = math.sin(self.x1[2] + self.z1[1])
        c1 = math.cos(self.x1[2] + self.z1[1])
        s2 = math.sin(self.x2[2] + self.z2[1])
        c2 = math.cos(self.x2[2] + self.z2[1])
        
        hat_e = self.x2 - self.x1 + np.array([
            self.z2[0]*c2 - self.z1[0]*c1,
            self.z2[0]*s2 - self.z1[0]*s1,
            self.z2[1] - self.z2[2] - self.z1[1] + self.z1[2]
        ])
        while hat_e[2] >= math.pi: hat_e[2] -= math.pi*2
        while hat_e[2] < - math.pi: hat_e[2] += math.pi*2
            
        Q1 = np.diag([(self.z1[0]*sensor_noise_rate[0])**2, sensor_noise_rate[1]**2, sensor_noise_rate[2]**2])
        R1 = - np.array([[c1, -self.z1[0]*s1, 0],
                        [s1, self.z1[0]*c1, 0],
                        [0, 1, -1]])
        
        Q2 = np.diag([(self.z2[0]*sensor_noise_rate[0])**2, sensor_noise_rate[1]**2, sensor_noise_rate[2]**2])
        R2 = np.array([[c2, -self.z2[0]*s2, 0],
                      [s2, self.z2[0]*c2, 0],
                      [0, 1, -1]])
        
        Sigma = R1.dot(Q1).dot(R1.T) + R2.dot(Q2).dot(R2.T)
        Omega = np.linalg.inv(Sigma)
        
        print(Sigma)

In [5]:
import itertools
def make_edges(hat_xs, zlist):
    landmark_keys_zlist = {}
    
    for step in zlist:
        for z in zlist[step]:
            landmark_id = z[0]
            if landmark_id not in landmark_keys_zlist:
                landmark_keys_zlist[landmark_id] = []
                
            landmark_keys_zlist[landmark_id].append((step, z))
            
    edges = []
    for landmark_id in landmark_keys_zlist:
        step_pairs = list(itertools.combinations(landmark_keys_zlist[landmark_id], 2))
        edges += [ObsEdge(xz1[0], xz2[0], xz1[1], xz2[1], hat_xs) for xz1, xz2 in step_pairs]
        
    return edges

In [6]:
hat_xs, zlist = read_data()
edges = make_edges(hat_xs, zlist)
draw(hat_xs, zlist, edges)

[[ 0.12467633  0.00761939 -0.00055506]
 [ 0.00761939  0.01672374  0.00879378]
 [-0.00055506  0.00879378  0.01      ]]
[[0.09311014 0.00340994 0.00024708]
 [0.00340994 0.01448072 0.00694825]
 [0.00024708 0.00694825 0.01      ]]
[[ 0.18880057 -0.10845566  0.00799577]
 [-0.10845566  0.24831637  0.01009167]
 [ 0.00799577  0.01009167  0.01      ]]
[[ 0.19363178 -0.12533378  0.00991096]
 [-0.12533378  0.36080862  0.0097708 ]
 [ 0.00991096  0.0097708   0.01      ]]
[[ 0.18772615 -0.08747217  0.00600869]
 [-0.08747217  0.15602513  0.01042921]
 [ 0.00600869  0.01042921  0.01      ]]
[[ 0.16442667 -0.0523178   0.00396599]
 [-0.0523178   0.08371387  0.00998019]
 [ 0.00396599  0.00998019  0.01      ]]
[[ 0.12493171 -0.01508265  0.00175495]
 [-0.01508265  0.03263742  0.00871455]
 [ 0.00175495  0.00871455  0.01      ]]
[[0.08458097 0.00564102 0.00021771]
 [0.00564102 0.01325754 0.00590746]
 [0.00021771 0.00590746 0.01      ]]
[[ 0.11391729 -0.00536151  0.00823362]
 [-0.00536151  0.25142837  0.005649

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …